<a href="https://colab.research.google.com/github/casalazara/Machine-Learning-IELE/blob/master/Reto%201/Reto_1_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Librerías a usar

In [ ]:
import pandas as pd #Manejo de los datos
from sklearn.model_selection import train_test_split #Separación de los datos
from sklearn.linear_model import LinearRegression #Modelo a usar
from sklearn.metrics import r2_score,mean_squared_error #Métricas


#Lectura del archivo

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/casalazara/Machine-Learning-IELE/master/Reto%201/Life_Expectancy_Data.csv')

In [ ]:
df.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


#Preparación de los datos


## Hot encoding columna Status
Se hace para pasar esta variable tipo object (String) a tipo numérica binaria añadiendo dos columnas con los dos valores posibles de esta variable

In [ ]:
status = pd.get_dummies(df.Status)
status.head()

,Developed,Developing
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1


## Unión con un DataFrame auxiliar y eliminación de la columnas columnas Status y Country
Se añaden las dos columnas binarias creadas apartir de los valores de la columna Status y se elimina la columna Status y Country pues necesitamos variables numéricas.

In [ ]:
aux = pd.concat([df, status], axis=1) #Combinación de los dos df, el auxiliar creado previamente y el cargado.

In [ ]:
aux.drop(columns=['Status','Country'],inplace=True)
aux

,Year,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Developed,Developing
0,2015,65.0,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1,0,1
1,2014,59.9,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0,0,1
2,2013,59.9,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9,0,1
3,2012,59.5,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8,0,1
4,2011,59.2,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,2004,44.3,723.0,27,4.36,0.000000,68.0,31,27.1,42,67.0,7.13,65.0,33.6,454.366654,12777511.0,9.4,9.4,0.407,9.2,0,1
2934,2003,44.5,715.0,26,4.06,0.000000,7.0,998,26.7,41,7.0,6.52,68.0,36.7,453.351155,12633897.0,9.8,9.9,0.418,9.5,0,1
2935,2002,44.8,73.0,25,4.43,0.000000,73.0,304,26.3,40,73.0,6.53,71.0,39.8,57.348340,125525.0,1.2,1.3,0.427,10.0,0,1
2936,2001,45.3,686.0,25,1.72,0.000000,76.0,529,25.9,39,76.0,6.16,75.0,42.1,548.587312,12366165.0,1.6,1.7,0.427,9.8,0,1


In [ ]:
aux.columns

Index(['Year', 'Life expectancy ', 'Adult Mortality', 'infant deaths',
       'Alcohol', 'percentage expenditure', 'Hepatitis B', 'Measles ', ' BMI ',
       'under-five deaths ', 'Polio', 'Total expenditure', 'Diphtheria ',
       ' HIV/AIDS', 'GDP', 'Population', ' thinness  1-19 years',
       ' thinness 5-9 years', 'Income composition of resources', 'Schooling',
       'Developed', 'Developing'],
      dtype='object')

##Checkeo de valores no nulos



In [ ]:
aux.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Year                             2938 non-null   int64  
 1   Life expectancy                  2928 non-null   float64
 2   Adult Mortality                  2928 non-null   float64
 3   infant deaths                    2938 non-null   int64  
 4   Alcohol                          2744 non-null   float64
 5   percentage expenditure           2938 non-null   float64
 6   Hepatitis B                      2385 non-null   float64
 7   Measles                          2938 non-null   int64  
 8    BMI                             2904 non-null   float64
 9   under-five deaths                2938 non-null   int64  
 10  Polio                            2919 non-null   float64
 11  Total expenditure                2712 non-null   float64
 12  Diphtheria          

Parece que no hay demasiados valores no numéricos. Sin embargo, los que hay pueden afectar el desempeño del modelo por lo que van a ser llenados por la media de cada columna ya que si se eliminan, se irían apróximadamente 1289 filas, una gran cantidad. Lo que haremos es tener en cuenta estas columnas con valores vacíos para los modelos que vamos a implementar y por ahora llenarlas con la media.

In [ ]:
aux.fillna(aux.mean(),inplace=True)
aux.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Year                             2938 non-null   int64  
 1   Life expectancy                  2938 non-null   float64
 2   Adult Mortality                  2938 non-null   float64
 3   infant deaths                    2938 non-null   int64  
 4   Alcohol                          2938 non-null   float64
 5   percentage expenditure           2938 non-null   float64
 6   Hepatitis B                      2938 non-null   float64
 7   Measles                          2938 non-null   int64  
 8    BMI                             2938 non-null   float64
 9   under-five deaths                2938 non-null   int64  
 10  Polio                            2938 non-null   float64
 11  Total expenditure                2938 non-null   float64
 12  Diphtheria          

## Normalización de los datos
Se hace para que el rango de las variables sea el mismo (entre 0 y 1) de manera que no haya variables que influencien más que otras por el rango además que ayuda a que el algoritmo funcione un poco más rápido.


In [ ]:
for columna in aux.columns:
  aux[columna] = (aux[columna] - aux[columna].min()) / (aux[columna].max() - aux[columna].min())
aux

,Year,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Developed,Developing
0,1.000000,0.544592,0.362881,0.034444,0.000000,0.003659,0.653061,0.005439,0.209733,0.0332,0.031250,0.452118,0.649485,0.000000,0.004889,0.026074,0.619565,0.603509,0.505274,0.487923,0.0,1.0
1,0.933333,0.447818,0.373961,0.035556,0.000000,0.003774,0.622449,0.002319,0.203940,0.0344,0.572917,0.453279,0.618557,0.000000,0.005127,0.000253,0.630435,0.610526,0.502110,0.483092,0.0,1.0
2,0.866667,0.447818,0.369806,0.036667,0.000000,0.003759,0.642857,0.002027,0.198146,0.0356,0.614583,0.450377,0.639175,0.000000,0.005287,0.024525,0.637681,0.617544,0.495781,0.478261,0.0,1.0
3,0.800000,0.440228,0.375346,0.038333,0.000000,0.004014,0.673469,0.013135,0.192352,0.0372,0.666667,0.473012,0.670103,0.000000,0.005608,0.002857,0.644928,0.628070,0.488397,0.473430,0.0,1.0
4,0.733333,0.434535,0.379501,0.039444,0.000000,0.000364,0.683673,0.014200,0.187717,0.0388,0.677083,0.435287,0.680412,0.000000,0.000519,0.002302,0.655797,0.635088,0.478903,0.458937,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2933,0.266667,0.151803,1.000000,0.015000,0.243561,0.000000,0.683673,0.000146,0.302433,0.0168,0.666667,0.392339,0.649485,0.663366,0.003799,0.009875,0.336957,0.326316,0.429325,0.444444,0.0,1.0
2934,0.200000,0.155598,0.988920,0.014444,0.226764,0.000000,0.061224,0.004703,0.297798,0.0164,0.041667,0.356936,0.680412,0.724752,0.003790,0.009764,0.351449,0.343860,0.440928,0.458937,0.0,1.0
2935,0.133333,0.161290,0.099723,0.013889,0.247480,0.000000,0.734694,0.001433,0.293163,0.0160,0.729167,0.357516,0.711340,0.786139,0.000467,0.000097,0.039855,0.042105,0.450422,0.483092,0.0,1.0
2936,0.066667,0.170778,0.948753,0.013889,0.095745,0.000000,0.765306,0.002493,0.288528,0.0156,0.760417,0.336042,0.752577,0.831683,0.004589,0.009558,0.054348,0.056140,0.450422,0.473430,0.0,1.0


#Modelos

Función general para todos los modelos, allí se parte el mismo conjunto de entrenamiento y de prueba para todos los modelos ya que el parámetro random_state es el mismo siempre. Luego, se crea el modelo y se ajusta a los datos de entrenamiento y se calculan las métricas correspondientes (R2 ajustado en el conjunto de entrenamiento y mse en el conjunto de prueba)

In [ ]:
def resultados(columnas,datos,objetivo='Life expectancy '):
  X_train, X_test, y_train, y_test = train_test_split(datos[columnas], datos[objetivo], random_state=0) #Por defecto se asigna 25% al conjunto de prueba y 75% al de entrenamiento
  #Creación y ajuste del modelo de regresión lineal
  reg = LinearRegression() 
  reg.fit(X_train, y_train)
  #Predicción y cálculo de las métricas solicitadas
  #R2 Ajustado
  prediction_y_train = reg.predict(X_train)
  r_squared = r2_score(y_train,prediction_y_train)
  K = X_train.shape[1]
  n = len(X_train)
  r_squared_fitted = r_squared-((K-1)/(n-K))*(1-r_squared)
  #MSE
  prediction_y_test = reg.predict(X_test)
  mse = mean_squared_error(y_test,prediction_y_test)
  return (f'Se obtuvo un r2 ajustado de {r_squared_fitted} en el conjunto de entrenamiento y un mean squared error de {mse} en el conjunto de prueba',reg)

##Modelo 1
Este modelo tendrá en cuenta todas las columnas.

In [ ]:
objetivo = 'Life expectancy '
lista_Columnas = aux.columns.to_list()
lista_Columnas.remove(objetivo)
colM1 = lista_Columnas
columnas = lista_Columnas

###Resultados

In [ ]:
Modelo1 = resultados(columnas,aux,objetivo)
Modelo1[0]

'Se obtuvo un r2 ajustado de 0.8142303961519466 en el conjunto de entrenamiento y un mean squared error de 0.005822845021164382 en el conjunto de prueba'

##Modelo 2
Este modelo no tendrá en cuenta las columnas:
1. Alcohol
2. Hepatitis B
3. BMI
4. Total expenditure
5. GDP
6. Population
7. thinness  1-19 years
8. thinness 5-9 years
9. Income composition of resources
10. Schooling

Pues son las 10 más vacías de las que llenamos con la media previamente.


In [ ]:
objetivo = 'Life expectancy '
lista_Columnas = aux.columns.to_list()
lista_Columnas.remove(objetivo)
a_remover = ['Alcohol','Hepatitis B',' BMI ','Total expenditure','GDP','Population',' thinness  1-19 years',' thinness 5-9 years','Income composition of resources', 'Schooling']
lista_Columnas = [x for x in lista_Columnas if x not in a_remover]
colM2 = lista_Columnas

columnas = lista_Columnas

###Resultados

In [ ]:
Modelo2 = resultados(columnas,aux,objetivo)
Modelo2[0]

'Se obtuvo un r2 ajustado de 0.7297738180169392 en el conjunto de entrenamiento y un mean squared error de 0.009385997777545074 en el conjunto de prueba'

##Modelo 3
Este modelo no tendrá en cuenta las columnas:

1. Income composition of resources
2. GDP
3. Total expenditure
4. Hepatitis B
5. Alcohol
6. Population
7. Schooling

Pues previamente contaban con una gran cantidad de valores nulos.

In [ ]:
objetivo = 'Life expectancy '
lista_Columnas = aux.columns.to_list()
lista_Columnas.remove(objetivo)
a_remover = ['Income composition of resources','GDP','Total expenditure','Hepatitis B','Alcohol','Population','Schooling']
lista_Columnas = [x for x in lista_Columnas if x not in a_remover]
colM3 = lista_Columnas

columnas = lista_Columnas

###Resultados

In [ ]:
Modelo3 = resultados(columnas,aux,objetivo)
Modelo3[0]

'Se obtuvo un r2 ajustado de 0.7643632137598634 en el conjunto de entrenamiento y un mean squared error de 0.008222205570559008 en el conjunto de prueba'

#Variable de mayor afectación


## Pesos de todas las variables
Dado que el modelo 1 fue el que mejor r2 ajustado y mse tuvo este será tomado como el modelo de referencia para identificar la variable de mayor afectación, a continuación emparejaremos las variables con los pesos que el modelo les asignó y las ordenaremos descendentemente.

In [ ]:
parejas = list(zip(colM1,Modelo1[1].coef_))
parejas.sort(key=lambda x:x[1],reverse=True)
parejas

[('infant deaths', 3.391790801562926),
 ('Schooling', 0.22344196366163807),
 ('Income composition of resources', 0.11254823371245773),
 ('GDP', 0.08789125244828759),
 (' BMI ', 0.07951091370471196),
 ('Diphtheria ', 0.07084676531183802),
 ('Polio', 0.05819857839541722),
 ('Total expenditure', 0.04017070069564396),
 ('Population', 0.023720944656703866),
 ('percentage expenditure', 0.02248923572642583),
 (' thinness 5-9 years', 0.018361479206929838),
 ('Alcohol', 0.017362200596462336),
 ('Developed', 0.014932698247758482),
 ('Year', -0.004192615434181024),
 ('Developing', -0.014932698247760279),
 ('Hepatitis B', -0.026916999546542305),
 (' thinness  1-19 years', -0.05285170334080163),
 ('Measles ', -0.09220403948606884),
 ('Adult Mortality', -0.27221021593181166),
 (' HIV/AIDS', -0.4320584503230993),
 ('under-five deaths ', -3.540296098243306)]

## Peso de la variable de mayor afectación 
Una vez tengamos las variables ordenadas descendentemente compararemos los valores absolutos de la variable de mayor peso y la de menor peso para identificar cuál es la de mayor afectación

In [ ]:
mas_pesada = parejas[0] if abs(parejas[0][1]) > abs(parejas[-1][1]) else parejas[-1]
mas_pesada

('under-five deaths ', -3.540296098243306)

Encontramos que la variable de mayor afectación son la cantidad de muertes de niños menores de 5 años con un peso negativo bastante grande en valor absoluto. Sin embargo analizando los pesos de las demás variables vemos que su valor absoluto está entre 0 y 1 y solo hay 2 que tienen un valor absoluto mayor a 3, ambas están relacionadas estrechamente con la mortandad infantil por lo que podríamos decir que la esperanza de vida de un país está fuertemente ligada a la mortandad infantil.

